In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Median

In [2]:
df = pd.read_excel("Commodities_Returns.xlsx", index_col="Date",parse_dates=True)

In [3]:
returns=df

In [4]:
!pip install arch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.4/983.4 kB 25.1 MB/s eta 0:00:00


In [6]:
from arch import arch_model

In [7]:
df1= pd.read_excel("Commodities Daily Realized Covariance (Neural).xlsx", sheet_name= "Vol", index_col="Date",parse_dates=True)


In [8]:
df1.iloc[int(len(df1)*0.7)]

CORN_USD      0.009164
NATGAS_USD    0.027221
SOYBN_USD     0.007867
WTICO_USD     0.022200
SUGAR_USD     0.017678
WHEAT_USD     0.014087
XAU_USD       0.007528
Name: 2015-11-12 00:00:00, dtype: float64

In [9]:
forecasts1=np.zeros([int(len(df1)*0.3),len(df.columns)])
test_size = int(len(df1)*0.3)
for i in range(len(df.columns)):
  am = arch_model(returns[df.columns[i]], vol="garch", p=1, o=0, q=1, dist="Normal")
  res = am.fit(update_freq=5,last_obs="2015-11-16")
  rolling_predictions = []
  for j in range(int(test_size/1)):
    train = returns[:-(test_size-j*1)]
    model = arch_model(train[df.columns[i]], p=1, o=0, q=1)
    model_fit = model.fit(disp='off')
    pred = model_fit.forecast(horizon=1)
    rolling_predictions.append(np.sqrt(pred.variance.values[-1,:]))
  rolling_predictions=np.sqrt(rolling_predictions)**2
  rolling_predictions=rolling_predictions.reshape(forecasts1.shape[0])
  forecasts1[:,i]=rolling_predictions/100

Iteration:      5,   Func. Count:     38,   Neg. LLF: 4933.365684271397
Iteration:     10,   Func. Count:     64,   Neg. LLF: 4923.080849461199
Optimization terminated successfully    (Exit mode 0)
            Current function value: 4923.080849459593
            Iterations: 10
            Function evaluations: 64
            Gradient evaluations: 10
Iteration:      5,   Func. Count:     34,   Neg. LLF: 6915.684776984679
Iteration:     10,   Func. Count:     63,   Neg. LLF: 5832.233863552377
Optimization terminated successfully    (Exit mode 0)
            Current function value: 5832.233602464265
            Iterations: 13
            Function evaluations: 77
            Gradient evaluations: 13
Iteration:      5,   Func. Count:     34,   Neg. LLF: 4388.3180234171605
Iteration:     10,   Func. Count:     62,   Neg. LLF: 4371.775478410747
Optimization terminated successfully    (Exit mode 0)
            Current function value: 4371.775440198395
            Iterations: 12
            Fu

In [10]:
df2=df1.iloc[int(len(df1)*0.7)+1:len(df1)]

In [11]:
for i in range(len(df2.columns)):
  df2[df2.columns[i]]=forecasts1[:,i]

<ipython-input-11-ae522ed45476>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2[df2.columns[i]]=forecasts1[:,i]


In [12]:
df2.to_excel("GARCH(1,1)-Norm forecasts.xlsx")